In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import requests
import time
import re

In [3]:
base_url = 'https://me.cleartrip.com'

In [4]:
def get_info(hotel_name):
    url = hotel_name.find_element_by_tag_name('a').get_attribute('href')
    
    browser.get(url)

In [79]:
browser = webdriver.Chrome('./chromedriver')

In [63]:
wait_obj = WebDriverWait(browser, 500)

In [6]:
res = requests.get('https://me.cleartrip.com/hotels/united-states/miami/')

In [7]:
num_pages = 38

In [8]:
soup = BeautifulSoup(res.text, 'lxml')

In [9]:
page_num_buttons = soup.select('.pagination a')

In [10]:
page_num_buttons[-1].get('href')

'/hotels/united-states/miami?page=2'

In [11]:
page_num_buttons = page_num_buttons[:-1]

In [12]:
page_num_buttons[-1].get('href')

'/hotels/united-states/miami?page=38'

In [13]:
base_url + page_num_buttons[0].get('href')

'https://me.cleartrip.com/hotels/united-states/miami?page=2'

In [14]:
page_num_hrefs = [(base_url + button.get('href')) for button in page_num_buttons]

In [15]:
# page_num_hrefs

In [16]:
# hotels on the first page

hotels_a_tag_list = soup.select('.hotels-name a')

In [17]:
hotels_a_tag_list[:3]

[<a href="/hotels/details/192017?c=200419|210419&amp;r=2,0" rel="nofollow" title="Hilton Bentley South Beach, Miami, United States"><span>Hilton Bentley South Beach</span></a>,
 <a href="/hotels/details/292714?c=200419|210419&amp;r=2,0" rel="nofollow" title="Kimpton EPIC Hotel, Miami, United States"><span>Kimpton EPIC Hotel</span></a>,
 <a href="/hotels/details/345087?c=200419|210419&amp;r=2,0" rel="nofollow" title="JW Marriott Marquis Miami, Miami, United States"><span>JW Marriott Marquis Miami</span></a>]

In [18]:
len(hotels_a_tag_list)

20

In [19]:
hotels_urls = [(base_url + hotel_a_tag.get('href')) for hotel_a_tag in hotels_a_tag_list]

In [20]:
len(hotels_urls)

20

In [21]:
hotels_urls[:3]

['https://me.cleartrip.com/hotels/details/192017?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/292714?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/345087?c=200419|210419&r=2,0']

In [22]:
for page_num_href in page_num_hrefs:
    res = requests.get(page_num_href)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    hotels_a_tag_list = soup.select('.hotels-name a')
    hotels_urls += [(base_url + hotel_a_tag.get('href')) for hotel_a_tag in hotels_a_tag_list]

In [23]:
len(hotels_urls)

755

In [24]:
hotels_urls[:6]

['https://me.cleartrip.com/hotels/details/192017?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/292714?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/345087?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/367313?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/367317?c=200419|210419&r=2,0',
 'https://me.cleartrip.com/hotels/details/367353?c=200419|210419&r=2,0']

In [73]:
def get_dynamic_info(hotel_url):
    browser.get(hotel_url)
    browser.maximize_window()
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    
    try:
        rating_elem = browser.find_element_by_css_selector('li.review span[rel="tTooltip"]')
        ActionChains(browser).move_to_element(rating_elem).perform()
        orig_title = rating_elem.get_attribute('original-title')
        rating = int(re.findall('TripAdvisor traveller rating, (.+)/5', orig_title)[0])
    except:
        rating = 'NA'

    try:
        num_reviews_elem = browser.find_element_by_class_name('reviewLink')
        num_reviews = int(re.findall('([0-9]+) reviews', num_reviews_elem.text)[0])
    except:
        num_reviews = 'NA'
    
    try:
        price_elem = price_elem = browser.find_element_by_css_selector('#b-min-price')
        price = price_elem.text
    except:
        price = 'NA'
        
    hotel_info = {}
    more_info = wait_obj.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.shwMoreDesc")))
#     more_info = browser.find_element_by_css_selector('a.shwMoreDesc')
    more_info.click()
    dynamic_info_area = browser.find_elements_by_css_selector('.amenitiesCategory div')[0]
    
    try: 
        # for pages with property location, rooms, etc. headings
        info = amenities_cat.find_elements_by_css_selector('p')[1:-1]
        
        # exception occurs if these headings not found
        property_location_heading = info[0].find_element_by_css_selector('b')
        rooms_heading = info[1].find_element_by_css_selector('b')
        amenities_heading = info[2].find_element_by_css_selector('b')
        dining_heading = info[3].find_element_by_css_selector('b')
        business_heading = info[4].find_element_by_css_selector('b')
        
        property_location = info[0].text.replace(property_location_heading, '').strip()
        rooms = info[1].text.replace(property_location_heading, '').strip()
        amenities = info[2].text.replace(property_location_heading, '').strip()
        dining = info[3].text.replace(property_location_heading, '').strip()
        business_other_amenities = info[4].text.replace(property_location_heading, '').strip()
        
        more_info = {
            'property_location': property_location,
            'rooms': rooms,
            'amenities': amenities,
            'dining': dining,
            'business_other_amenities': busines_other_amenities
        }
        
    except:
        # when headings are not present
        more_info = dynamic_info_area.text.strip()
    
    dynamic_info = {
        'rating': rating,
        'num_reviews': num_reviews,
        'price': price,
        'hotel_info': more_info
        
    }
    
    return dynamic_info

In [80]:
get_dynamic_info(hotels_urls[2])

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=43975): Max retries exceeded with url: /session/05d47ca246fdca2c179228938259e788/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6efe8c83c8>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [77]:
hotels_urls[2]

'https://me.cleartrip.com/hotels/details/345087?c=200419|210419&r=2,0'

In [57]:
all_hotels = []
for hotel_url in hotels_urls:
    with open('log', 'a') as logfile:
        logfile.write(hotel_url+'\n')
    res = requests.get(hotel_url)
    soup = BeautifulSoup(res.text, 'lxml')
    
    dynamic_info = get_dynamic_info(hotel_url)
    print(dynamic_info)
    
    hotel_info = {}
    
    hotel_name = soup.select('h1')[0].getText().strip()
    hotel_address = soup.select('h1 small')[0].getText().strip()
    
    amenities_cats_except_first = soup.select('div.amenitiesCategory')[1:]
    
    quick_facts_ul = soup.select('.clearFix.hotelStats')[0]
    quick_facts_li = quick_facts_ul.find_all('li')
    
    for quick_fact in quick_facts_li:
        heading = quick_facts_li[0].find('small').text.strip()
        hotel_info[heading] = quick_facts_li[0].find('span').text.strip()
    
    # making dictionary from information gathered till now
    
    hotel_info = {
        'hotel_name': hotel_name,
        'hotel_address': hotel_address,
    }
    
    hotel_info = dict(list(hotel_info.items()) + list(dynamic_info.items()))
    
    # scraping other information
    amenities_cats_except_second = amenities_cats_except_first[1:]
    
    for amenities_cat in amenities_cats_except_second:
        heading = amenities_cat.find('strong').text
        hotel_info[heading] = amenities_cat.find('ul').text.strip().split('\n')
        
    all_hotels += hotel_info

WebDriverException: Message: unknown error: Element <a href="javascript:void(0)" class="shwMoreDesc">...</a> is not clickable at point (522, 7). Other element would receive the click: <div class="stickyHotelTitle row">...</div>
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=2.40.565383 (76257d1ab79276b2d53ee976b2c3e3b9f335cde7),platform=Linux 4.15.0-47-generic x86_64)


In [42]:
print(all_hotels)

['hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'Business Services', 'Basics', 'Recreation', 'Travel', 'Personal Services', 'hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'General', 'Food & Beverage', 'Business Services', 'Front Desk Services', 'Travel', 'Recreation', 'Kids', 'Pets', 'hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'General', 'Food & Beverage', 'Business Services', 'Front Desk Services', 'Travel', 'Recreation', 'Kids', 'hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'Basics', 'Recreation', 'Travel', 'Personal Services', 'No Category', 'hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'Basics', 'Personal Services', 'Recreation', 'Travel', 'hotel_name', 'hotel_address', 'rating', 'num_reviews', 'price', 'hotel_info', 'Food & Beverage', 'Busin